# Limpieza, exploración y preparación de datos prueba PLANEA 2015

## Usado también como caso para la materia Programación para Analítica de Datos

#### Mtra. Gisel Hernández Chávez

### Contenido

1. Cargar base de datos PLANEA 2015 y explorar información general. ESTE ARCHIVO CONTIENE FILAS QUE SE ADICIONARON AL ORIGINAL CSV CON PREPARATORIAS QUE NO HICIERON PLANEA O EMISORES DE CERTIFICADOS DE PREPARATORIA (REVALIDACIONES Y ACREDITACIONES), ASÍ COMO DE ESCUELAS CLASIFICADAS COMO EXTRANJERAS.
2. Procesar valores faltantes

    2.1 Imputar localidades faltantes con nombre del municipio
    
    2.2 Modificar ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO convirtiendo a booleano
    
    2.3 Limpieza de columna EXTENSIÓN 
    
3. Eliminar CCT duplicados con menor ind_gral 
4. Imputar con 0 donde aparece 'S/R'
5. Guardar archivo de PLANEA 2015 limpio y sus tipos
6. Reporte EDA con pandas-profiling


### Análisis de posibilidades de nuevas características (predictoras):

+ Los resultados de las prepas en la prueba PLANEA en 2015 por preparatoria se usarán para crear un indicador de desempeño "ind_integral" que será la suma de "ind-len" e "ind-mat". Estos son una suma ponderada:

    ind_len = 0.25*esc_leng_I + 0.5*esc_leng_II + 0.75*esc_leng_III + 1*esc_leng_IV
    
    ind_mat = 0.25*esc_mat_I + 0.5*esc_mat_II + 0.75*esc_mat_III + 1*esc_mat_IV
    
    Esta nueva característica será del tipo razón y se probará en la regresión.
    
+ La clasificación de las prepas según su subsistema no se agregará porque ya se tiene una clasificación parecida [pública/privada]:
    + ASOCIACIÓN CIVIL
    + BACHILLERATO AUTÓNOMO
    + CECYTE
    + COBACH
    + CONALEP
    + DGECYTM
    + DGETA
    + DGETI
    + EMSAD
    + OTRAS FEDERALES
    + PARTICULARES
    + TELEBACHILLERATOS
+ La clasificación de las prepas según su sostenimiento no se agregará porque ya se tiene una clasificación parecida [pública/privada]:
    + particulares
    + federal
    + autónoma
    + estatal
+ La clasificación de las localidades de las prepas en cuanto al grado de marginación se agregará como característica categórica. Se convertirá en ordinal con valores del 1 al 5 

    En PLANEA se divide en:
    + MUY ALTO, ALTO, MEDIO, BAJO y MUY BAJO

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
#import pandas_profiling
#from pandas_profiling import ProfileReport

In [4]:
# Para obtener múltiples salidas de una celda
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 1. Cargar base de datos con PLANEA y explorar información general

+ Se especifican datos de subsistema, sostenimiento y grado de marginación, además de los resultados de la prueba y algunos resultados del análisis exploratorio

In [5]:
#tray = "C:\\Users\\ghernand\\Documents\\Desercion\\data\\"
planea = pd.read_csv("NAC_ESCUELAS_PMS2015.csv", low_memory=False)
planea

FileNotFoundError: [Errno 2] No such file or directory: 'NAC_ESCUELAS_PMS2015.csv'

In [ ]:
# Respaldo
planea_bk = planea.copy()

In [ ]:
planea.head(2)  # dos primeras filas
planea.tail(2)  # dos últimas filas

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
0,12,UNIDAD ACADÉMICA: PREPARATORIA NO. 36,12UBH0009Z,MATUTINO,NaN,GUERRERO,EDUARDO NERI,ZUMPANGO DEL RIO,BACHILLERATO AUTÓNOMO,ALTO,...,22.6,9.5,3.2,27.6,6.4,34.0,3.8,47.60,35.775,83.375
1,1,CBTA NO. 40 JOSÉ GUADALUPE POSADA,01DTA0040J,MATUTINO,NaN,AGUASCALIENTES,ASIENTOS,VILLA JUAREZ,DGETA,BAJO,...,30,12.8,7.1,44.5,16.9,61.4,16.5,56.55,43.850,100.400


,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
14652,33,SISTEMA EDUCATIVO NACIONAL *10343,11DEX0001G,MATUTINO,NaN,GUANAJUATO,CELAYA,NaN,PÚBLICO,BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14653,33,ESCUELA REGIONAL DE EDUCACION MEDIA SUPERIOR D...,14UBH0066P,MATUTINO,NaN,JALISCO,OCOTLAN,NaN,PÚBLICO,MUY BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Tipo de cada columna
planea.dtypes

CLAVE DE LA ENTIDAD                                        int64
NOMBRE_ESCUELA                                            object
CLAVE_ESCUELA                                             object
TURNO                                                     object
EXTENSIÓN                                                 object
ENTIDAD                                                   object
MUNICIPIO                                                 object
LOCALIDAD                                                 object
SUBSISTEMA                                                object
GRADO_MARGINACIÓN_LOCALIDAD_CCT                           object
SOSTENIMIENTO                                             object
ALUMNOS_PROGRAMADOS                                        int64
ALUMNOS_EVALUADOS                                          int64
PORCENTAJE_ ALUMNOS_EVALUADOS                            float64
ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO                      object
ALUMNOS_CONTESTARON_ 50_P

In [ ]:
# (filas, columnas)
planea.shape

(14654, 48)

In [ ]:
# Conteo de no nulos y tipos de cada columna
planea.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14654 entries, 0 to 14653
Data columns (total 48 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   CLAVE DE LA ENTIDAD                                    14654 non-null  int64  
 1   NOMBRE_ESCUELA                                         14654 non-null  object 
 2   CLAVE_ESCUELA                                          14654 non-null  object 
 3   TURNO                                                  14654 non-null  object 
 4   EXTENSIÓN                                              333 non-null    object 
 5   ENTIDAD                                                14654 non-null  object 
 6   MUNICIPIO                                              14654 non-null  object 
 7   LOCALIDAD                                              14513 non-null  object 
 8   SUBSISTEMA                                    

In [ ]:
# Lista de columnas
planea.columns

Index(['CLAVE DE LA ENTIDAD', 'NOMBRE_ESCUELA', 'CLAVE_ESCUELA', 'TURNO',
       'EXTENSIÓN', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'SUBSISTEMA',
       'GRADO_MARGINACIÓN_LOCALIDAD_CCT', 'SOSTENIMIENTO',
       'ALUMNOS_PROGRAMADOS', 'ALUMNOS_EVALUADOS',
       'PORCENTAJE_ ALUMNOS_EVALUADOS', 'ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO',
       'ALUMNOS_CONTESTARON_ 50_PORCIENTO_O_MÁS_PREG_LENGUAJE',
       'ALUMNOS_CONTESTARON_50_PORCIENTO_O_MÁS_PREG_MAT', 'esc_leng_I',
       'esc_leng_II', 'esc_leng_III', 'esc_leng_IV', 'ent_ss_mar_len_I',
       'ent_ss_mar_len_II', 'ent_ss_mar_len_III', 'ent_ss_mar_len_IV',
       'pa_ss_mar_len_I', 'pa_ss_mar_len_II', 'pa_ss_mar_len_III',
       'pa_ss_mar_len_IV', 'esc_mat_I', 'esc_mat_II', 'esc_mat_III',
       'esc_mat_IV', 'ent_ss_mar_mat_I', 'ent_ss_mar_mat_II',
       'ent_ss_mar_mat_III', 'ent_ss_mar_mat_IV', 'pa_ss_mar_mat_I',
       'pa_ss_mar_mat_II', 'pa_ss_mar_mat_III', 'pa_ss_mar_mat_IV', 'ind_len',
       'ind_mat', 'ind-compuesto', 'ind

In [ ]:
f'Total de escuelas evaluadas + adicionadas: {planea.shape[0]}  Cantidad de columnas de datos: {planea.shape[1]}'

'Total de escuelas evaluadas + adicionadas: 14654  Cantidad de columnas de datos: 48'

In [ ]:
# Localiza una fila dado su índice
planea.iloc[28]

CLAVE DE LA ENTIDAD                                                        1
NOMBRE_ESCUELA                                           CEMSAD LOS NEGRITOS
CLAVE_ESCUELA                                                     01EMS0015Z
TURNO                                                             VESPERTINO
EXTENSIÓN                                                                NaN
ENTIDAD                                                       AGUASCALIENTES
MUNICIPIO                                                     AGUASCALIENTES
LOCALIDAD                                                       LOS NEGRITOS
SUBSISTEMA                                                             EMSAD
GRADO_MARGINACIÓN_LOCALIDAD_CCT                                         BAJO
SOSTENIMIENTO                                                        ESTATAL
ALUMNOS_PROGRAMADOS                                                       31
ALUMNOS_EVALUADOS                                                         28

In [ ]:
# Localiza un conjunto de filas usando slicing
planea.iloc[2:10:2]

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
2,12,UNIDAD ACADÉMICA: PREPARATORIA NO. 36,12UBH0009Z,VESPERTINO,NaN,GUERRERO,EDUARDO NERI,ZUMPANGO DEL RIO,BACHILLERATO AUTÓNOMO,ALTO,...,22.6,9.5,3.2,7.2,3.5,10.7,1.8,33.1,29.825,62.925
4,1,TELEBACHILLERATO VALLADOLID,01ETH0016B,MATUTINO,NaN,AGUASCALIENTES,JESUS MARIA,VALLADOLID,TELEBACHILLERATOS,MUY BAJO,...,26.9,6.1,1.2,30.0,5.0,35.0,5.0,45.0,36.250,81.250
6,15,CENTRO MARIA MONTESSORI,15PCT0685F,MATUTINO,NaN,MÉXICO,ZUMPANGO,ZUMPANGO DE OCAMPO,PARTICULARES,BAJO,...,30.1,14.2,7.5,20.0,18.2,38.2,19.1,42.5,38.650,81.150
8,1,CEMSAD STA. MARÍA DE LA PAZ,01EMS0011C,VESPERTINO,NaN,AGUASCALIENTES,COSIO,SANTA MARIA DE LA PAZ,EMSAD,MEDIO,...,27,10.8,7.7,0.0,0.0,0.0,0.0,30.0,27.500,57.500


In [ ]:
# Ejemplo de creación de un DataFrame filtrando por una condición

matutino = planea.loc[planea['TURNO'] == 'MATUTINO']
matutino.drop(['TURNO'],axis=1,inplace=True)
# matutino = matutino.drop(['TURNO'],axis=1)
matutino.shape
matutino[['CLAVE DE LA ENTIDAD', 'NOMBRE_ESCUELA', 'CLAVE_ESCUELA', 
       'EXTENSIÓN', 'ENTIDAD', 'MUNICIPIO']]

C:\Users\ghernand\AppData\Local\Temp\ipykernel_5108\4239899522.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matutino.drop(['TURNO'],axis=1,inplace=True)


(10941, 47)

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,EXTENSIÓN,ENTIDAD,MUNICIPIO
0,12,UNIDAD ACADÉMICA: PREPARATORIA NO. 36,12UBH0009Z,NaN,GUERRERO,EDUARDO NERI
1,1,CBTA NO. 40 JOSÉ GUADALUPE POSADA,01DTA0040J,NaN,AGUASCALIENTES,ASIENTOS
3,12,CENTRO DE ESTUDIOS DEL BACHILLERATO GENERAL JE...,12PBH0012B,NaN,GUERRERO,EDUARDO NERI
4,1,TELEBACHILLERATO VALLADOLID,01ETH0016B,NaN,AGUASCALIENTES,JESUS MARIA
5,15,BACHILLERATO TECNOLOGICO DEL INSTITUTO EDUCATI...,15PCT0813K,NaN,MÉXICO,ZUMPANGO
...,...,...,...,...,...,...
14649,33,SISTEMA EDUCATIVO NACIONAL *10344,13DEX0001E,NaN,HIDALGO,PACHUCA DE SOTO
14650,33,INSTITUTO SONORENSE DE EDUCACION * 060911,26EEX0001H,NaN,SONORA,HERMOSILLO
14651,33,COLEGIO CALMECAC *9683,14PBJ0010Q,NaN,JALISCO,ZAPOPAN
14652,33,SISTEMA EDUCATIVO NACIONAL *10343,11DEX0001G,NaN,GUANAJUATO,CELAYA


### Observaciones:

+ Las columnas 'ind_len', 'ind_mat', 'ind-compuesto', 'ind_IV', 'ind_gral_len', 'ind_gral_mat','ind_gral'  son nuevas características creadas con los fines de la predicción.
+ El significado de las columnas se detalla a continuación:

## 2. Valores faltantes

In [ ]:
planea.isnull().sum()

CLAVE DE LA ENTIDAD                                          0
NOMBRE_ESCUELA                                               0
CLAVE_ESCUELA                                                0
TURNO                                                        0
EXTENSIÓN                                                14321
ENTIDAD                                                      0
MUNICIPIO                                                    0
LOCALIDAD                                                  141
SUBSISTEMA                                                   0
GRADO_MARGINACIÓN_LOCALIDAD_CCT                              0
SOSTENIMIENTO                                                0
ALUMNOS_PROGRAMADOS                                          0
ALUMNOS_EVALUADOS                                            0
PORCENTAJE_ ALUMNOS_EVALUADOS                                0
ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO                       107
ALUMNOS_CONTESTARON_ 50_PORCIENTO_O_MÁS_PREG_LENGUAJE  

## Hallazgos:

+ La mayoría de las escuelas no tiene extensiones. Explorar la columna para determinar si se elimina o se imputan los nulos.
+ Revisar filas sin localidad e imputar con Municipio o buscar una por una en internet
+ Revisar la única fila que no tiene ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO

### 2.1 Imputar localidades faltantes con nombre del municipio

In [ ]:
# Muestra una vista de las filas con valor nulo en LOCALIDAD
planea.loc[planea.LOCALIDAD.isnull()]

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
6613,16,CENTRO DE BACHILLERATO TECNOLOGICO AGROPECUARI...,16DTA0290R,MATUTINO,NaN,MICHOACAN,ZAMORA,NaN,DGETA,ALTO,...,26.9,11.4,6.3,100.0,100.0,200.0,200.0,100.000,100.0,200.000
6614,16,COLEGIO DE BACHILLERES MORELIA,16ECB5005V,MATUTINO,NaN,MICHOACAN,MORELIA,NaN,COBACH,MUY BAJO,...,32.6,13.1,6.7,40.0,20.0,60.0,20.0,50.000,45.0,95.000
6615,16,COLEGIO DE BACHILLERES ZAMORA,16ECB5002Y,MATUTINO,NaN,MICHOACAN,ZAMORA,NaN,COBACH,BAJO,...,28.4,11.2,5.1,9.1,10.0,19.1,19.1,36.375,42.5,78.875
6616,16,CECYTE EMSAD 51 EL PUERTO DE JUNGAPEO,16EMS0051F,MATUTINO,NaN,MICHOACAN,JUNGAPEO,NaN,EMSAD,MEDIO,...,27,10.8,7.7,50.0,0.0,50.0,0.0,50.000,25.0,75.000
6618,16,COLEGIO DE BACHILLERES LAZARO CARDENAS,16ECB5007T,MATUTINO,NaN,MICHOACAN,LAZARO CARDENAS,NaN,COBACH,MUY BAJO,...,32.6,13.1,6.7,20.0,0.0,20.0,0.0,45.000,30.0,75.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14649,33,SISTEMA EDUCATIVO NACIONAL *10344,13DEX0001E,MATUTINO,NaN,HIDALGO,PACHUCA DE SOTO,NaN,PÚBLICO,BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.000,0.0,0.000
14650,33,INSTITUTO SONORENSE DE EDUCACION * 060911,26EEX0001H,MATUTINO,NaN,SONORA,HERMOSILLO,NaN,PÚBLICO,MUY BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.000,0.0,0.000
14651,33,COLEGIO CALMECAC *9683,14PBJ0010Q,MATUTINO,NaN,JALISCO,ZAPOPAN,NaN,PARTICULARES,MUY BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.000,0.0,0.000
14652,33,SISTEMA EDUCATIVO NACIONAL *10343,11DEX0001G,MATUTINO,NaN,GUANAJUATO,CELAYA,NaN,PÚBLICO,BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.000,0.0,0.000


In [ ]:
# Localizar y reasignar valor en el mismo paso
planea.loc[planea.LOCALIDAD.isnull(),['LOCALIDAD']] = planea.MUNICIPIO  # Se imputa con valor de MUNICIPIO
planea.loc[planea.LOCALIDAD.isnull()]

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral


In [ ]:
# Vista de filas donde MUNICIPIO = LOCALIDAD
# Localizar y mostrar en un solo paso
planea.loc[planea.MUNICIPIO == planea.LOCALIDAD,['MUNICIPIO','LOCALIDAD']]

,MUNICIPIO,LOCALIDAD
9,ZUMPANGO,ZUMPANGO
12,SAN JOSE DE GRACIA,SAN JOSE DE GRACIA
14,SAN FRANCISCO DE LOS ROMO,SAN FRANCISCO DE LOS ROMO
17,RINCON DE ROMOS,RINCON DE ROMOS
18,RINCON DE ROMOS,RINCON DE ROMOS
...,...,...
14649,PACHUCA DE SOTO,PACHUCA DE SOTO
14650,HERMOSILLO,HERMOSILLO
14651,ZAPOPAN,ZAPOPAN
14652,CELAYA,CELAYA


### 2.2 Modificar ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO convirtiendo a booleano

In [ ]:
# Vista de aquellos donde ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO es null
planea.loc[planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO.isnull(), ['NOMBRE_ESCUELA','MUNICIPIO','LOCALIDAD',
                                                                  'ALUMNOS_PROGRAMADOS','ALUMNOS_EVALUADOS',
                                                                  'ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO']]

,NOMBRE_ESCUELA,MUNICIPIO,LOCALIDAD,ALUMNOS_PROGRAMADOS,ALUMNOS_EVALUADOS,ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO
10433,INSTITUTO TECNOLÓGICO Y DE ESTUDIOS SUPERIORES...,FRESNILLO,FRESNILLO,19,18,NaN
14548,RESOLUCION DE EQUIVALENCIA *10102,EXTRANJERO,EXTRANJERO,0,0,NaN
14549,RESOLUCION DE REVALIDACION *10101,EXTRANJERO,EXTRANJERO,0,0,NaN
14550,RESOLUCION DE REVALIDACION *9635,EXTRANJERO,EXTRANJERO,0,0,NaN
14551,SISTEMA EDUCATIVO NACIONAL *10336,GUADALAJARA,GUADALAJARA,0,0,NaN
...,...,...,...,...,...,...
14649,SISTEMA EDUCATIVO NACIONAL *10344,PACHUCA DE SOTO,PACHUCA DE SOTO,0,0,NaN
14650,INSTITUTO SONORENSE DE EDUCACION * 060911,HERMOSILLO,HERMOSILLO,0,0,NaN
14651,COLEGIO CALMECAC *9683,ZAPOPAN,ZAPOPAN,0,0,NaN
14652,SISTEMA EDUCATIVO NACIONAL *10343,CELAYA,CELAYA,0,0,NaN


In [ ]:
planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO.unique()

array([' ', 'SÍ', nan], dtype=object)

In [ ]:
planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO.value_counts(dropna=False)

       12936
SÍ      1611
NaN      107
Name: ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO, dtype: int64

In [ ]:
# Para obtener las proporciones
planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO.value_counts(dropna=False,normalize=True)

       0.882762
SÍ     0.109936
NaN    0.007302
Name: ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO, dtype: float64

In [ ]:
# Vista de aquellos donde ALUMNOS_EVALUADOS_MENOR_80_PORCIENT es ' ' (cadena vacía)
planea.loc[planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO ==' ', ['NOMBRE_ESCUELA','MUNICIPIO','LOCALIDAD',
                                                                  'ALUMNOS_PROGRAMADOS','ALUMNOS_EVALUADOS',
                                                                  'ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO']]

,NOMBRE_ESCUELA,MUNICIPIO,LOCALIDAD,ALUMNOS_PROGRAMADOS,ALUMNOS_EVALUADOS,ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO
0,UNIDAD ACADÉMICA: PREPARATORIA NO. 36,EDUARDO NERI,ZUMPANGO DEL RIO,122,112,
1,CBTA NO. 40 JOSÉ GUADALUPE POSADA,ASIENTOS,VILLA JUAREZ,224,218,
2,UNIDAD ACADÉMICA: PREPARATORIA NO. 36,EDUARDO NERI,ZUMPANGO DEL RIO,63,57,
3,CENTRO DE ESTUDIOS DEL BACHILLERATO GENERAL JE...,EDUARDO NERI,ZUMPANGO DEL RIO,5,5,
4,TELEBACHILLERATO VALLADOLID,JESUS MARIA,VALLADOLID,20,20,
...,...,...,...,...,...,...
14542,COLEGIO DE BACHILLERES PLANTEL APOZOL,APOZOL,APOZOL,40,39,
14544,JAIME TORRES BODET,MORELIA,MORELIA,9,9,
14545,ESCUELA DE ENFERMERIA,MORELIA,MORELIA,29,26,
14546,EMSAD APIZOLAYA,MAZAPIL,APIZOLAYA,8,8,


In [ ]:
# Usar dropna=False para mostrar conteo de nulos
planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO.value_counts(dropna=False)

       12936
SÍ      1611
NaN      107
Name: ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO, dtype: int64

In [ ]:
#pd.value_counts?

In [1]:
# Aquí se hace la imputación
d = {'SÍ': True, ' ': False, np.nan: False}

planea['ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO'] = planea['ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO'].map(d)


NameError: name 'np' is not defined

In [28]:
planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO

0        False
1        False
2        False
3        False
4        False
         ...  
14649      NaN
14650      NaN
14651      NaN
14652      NaN
14653      NaN
Name: ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO, Length: 14654, dtype: object

In [29]:
# Verificar que los que evaluaron menos del 80% de los programados sean
#los que tienen True en ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO
porc_80 = planea['ALUMNOS_EVALUADOS'] /planea['ALUMNOS_PROGRAMADOS']
planea.loc[ (porc_80 *100 < 80) & (planea['ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO'] != True ),['NOMBRE_ESCUELA',
                               'ALUMNOS_PROGRAMADOS','ALUMNOS_EVALUADOS',
                               'ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO' ]]

,NOMBRE_ESCUELA,ALUMNOS_PROGRAMADOS,ALUMNOS_EVALUADOS,ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO


In [30]:
# Usar dropna=False para mostrar conteo de nulos
planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO.value_counts(dropna=False)

False    12936
True      1611
NaN        107
Name: ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO, dtype: int64

In [31]:
planea.loc[planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO.isnull(), ['ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO']] = False

In [32]:
planea.loc[planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO.isnull(), ['NOMBRE_ESCUELA','MUNICIPIO','LOCALIDAD',
                                                                  'ALUMNOS_PROGRAMADOS','ALUMNOS_EVALUADOS',
                                                                  'ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO']]

,NOMBRE_ESCUELA,MUNICIPIO,LOCALIDAD,ALUMNOS_PROGRAMADOS,ALUMNOS_EVALUADOS,ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO


In [33]:
planea.ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO.value_counts()

False    13043
True      1611
Name: ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO, dtype: int64

### 2.3 Limpieza de columna EXTENSIÓN

In [34]:
planea.EXTENSIÓN.value_counts(dropna=False)
# 333 registros que repiten el CCT (extensiones)
# Se van a eliminar para no tener CCT duplicados

NaN             14321
EXTENSIÓN 1       246
EXTENSIÓN 2        47
EXTENSIÓN 3        14
EXTENSIÓN 4         7
EXTENSIÓN 5         3
EXTENSIÓN 6         2
EXTENSIÓN 7         1
EXTENSIÓN 48        1
TECARIO             1
EXTENSIÓN 26        1
EXTENSIÓN 11        1
EXTENSIÓN 14        1
EXTENSIÓN 9         1
EXTENSIÓN 25        1
EXTENSIÓN 22        1
EXTENSIÓN 21        1
EXTENSIÓN 19        1
EXTENSIÓN 10        1
EXTENSIÓN 17        1
EXTENSIÓN 8         1
Name: EXTENSIÓN, dtype: int64

### Hallazgo

+ Es raro el nombre de extensión 'TECARIO'

In [35]:
planea.loc[planea.EXTENSIÓN == 'TECARIO'] 

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
6019,16,CBTA N0. 69,16DTA0069Q,MATUTINO,TECARIO,MICHOACAN,TACAMBARO,SIN AGUA,DGETA,ALTO,...,26.9,11.4,6.3,29.6,29.6,59.2,11.1,45.35,49.025,94.375


In [36]:
planea.loc[planea.EXTENSIÓN == 'TECARIO',['EXTENSIÓN']] = np.NaN
planea.EXTENSIÓN.value_counts(dropna =False)

NaN             14322
EXTENSIÓN 1       246
EXTENSIÓN 2        47
EXTENSIÓN 3        14
EXTENSIÓN 4         7
EXTENSIÓN 5         3
EXTENSIÓN 6         2
EXTENSIÓN 9         1
EXTENSIÓN 7         1
EXTENSIÓN 48        1
EXTENSIÓN 26        1
EXTENSIÓN 11        1
EXTENSIÓN 14        1
EXTENSIÓN 25        1
EXTENSIÓN 22        1
EXTENSIÓN 21        1
EXTENSIÓN 19        1
EXTENSIÓN 10        1
EXTENSIÓN 17        1
EXTENSIÓN 8         1
Name: EXTENSIÓN, dtype: int64

### Remplazar NaN en EXTENSIÓN con cadena vacía ''

In [37]:
# ESTO ES LO QUE REALMENTE COLOCA LA CADENA VACÍA
planea['EXTENSIÓN'] = planea['EXTENSIÓN'].fillna('NO')

In [38]:
planea.EXTENSIÓN.value_counts()

NO              14322
EXTENSIÓN 1       246
EXTENSIÓN 2        47
EXTENSIÓN 3        14
EXTENSIÓN 4         7
EXTENSIÓN 5         3
EXTENSIÓN 6         2
EXTENSIÓN 9         1
EXTENSIÓN 7         1
EXTENSIÓN 48        1
EXTENSIÓN 26        1
EXTENSIÓN 11        1
EXTENSIÓN 14        1
EXTENSIÓN 25        1
EXTENSIÓN 22        1
EXTENSIÓN 21        1
EXTENSIÓN 19        1
EXTENSIÓN 10        1
EXTENSIÓN 17        1
EXTENSIÓN 8         1
Name: EXTENSIÓN, dtype: int64

planea['EXTENSIÓN'] == planea['EXTENSIÓN'].replace(np.nan, '', regex=True)

NO FUNCIONA

In [39]:
# Sin NaN
planea.CLAVE_ESCUELA.value_counts()

31DTA0013G    8
02ETC0004A    6
12DTA0005J    5
02ETC0014H    5
14ETC0010Q    5
             ..
15EBH0351S    1
13PBH3623V    1
18PBH0072K    1
15EBH0452Q    1
14UBH0066P    1
Name: CLAVE_ESCUELA, Length: 12857, dtype: int64

### Hallazgo

+ La clave (CCT) de una escuela se repite de 1 a 8 veces: de 1 a 8 registros con la misma clave

In [40]:
f'Total de claves de escuelas de las {planea.shape[0]} escuelas evaluadas o adicionadas: {len(planea.CLAVE_ESCUELA.unique())}'

'Total de claves de escuelas de las 14654 escuelas evaluadas o adicionadas: 12857'

In [41]:
planea.loc[planea.CLAVE_ESCUELA =='31DTA0013G'] 

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
14045,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 7,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,26.1,4.2,30.3,8.7,47.825,35.425,83.250
14047,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 1,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,22.6,3.2,25.8,4.3,44.125,33.800,77.925
14048,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 4,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,36.4,0.0,36.4,0.0,47.775,29.550,77.325
14049,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 2,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,14.3,4.8,19.1,0.0,38.100,34.575,72.675
14050,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 6,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,23.5,0.0,23.5,0.0,39.700,32.350,72.050
14051,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 8,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,14.6,2.1,16.7,4.2,40.125,30.775,70.900
14053,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 3,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,15.4,0.0,15.4,0.0,32.700,28.700,61.400
14054,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 5,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,14.3,0.0,14.3,0.0,35.725,25.000,60.725


### Hallazgo:
+ Este es un caso con EXTENSIÓN, sin un registro con la columna EXTENSIÓN  con valor NaN (no hay escuela matriz)
+ Es por ello que no podemos borrar las filas de extensiones

In [42]:
planea.iloc[8786]

CLAVE DE LA ENTIDAD                                                                  20
NOMBRE_ESCUELA                                                  JOSE MA MORELOS Y PAVON
CLAVE_ESCUELA                                                                20PBH0054J
TURNO                                                                        VESPERTINO
EXTENSIÓN                                                                            NO
ENTIDAD                                                                          OAXACA
MUNICIPIO                                                     SANTA MARIA HUAZOLOTITLAN
LOCALIDAD                                                JOSE MARIA MORELOS (POZA VERDE
SUBSISTEMA                                                                 PARTICULARES
GRADO_MARGINACIÓN_LOCALIDAD_CCT                                                    ALTO
SOSTENIMIENTO                                                              PARTICULARES
ALUMNOS_PROGRAMADOS             

In [43]:
# Para chequear si tiene valores nulos
planea['EXTENSIÓN'].isnull().values.any()


False

In [44]:
# Para contar NaN en una columna
planea['EXTENSIÓN'].isnull().sum()

0

In [45]:
# Para mostrar con EXTENSIÓN 1
planea.loc[planea.EXTENSIÓN =='EXTENSIÓN 1']

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
303,2,URBIVILLA,02ETC0006Z,VESPERTINO,EXTENSIÓN 1,BAJA CALIFORNIA,TIJUANA,TIJUANA,CECYTE,MUY BAJO,...,32.8,11.4,3.8,27.7,4.9,32.6,4.9,44.750,32.125,76.875
351,2,PRIMO TAPIA ADULTOS,02ETC0016F,VESPERTINO,EXTENSIÓN 1,BAJA CALIFORNIA,PLAYAS DE ROSARITO,PRIMO TAPIA,CECYTE,BAJO,...,31,12.8,5.7,3.0,70.6,73.6,11.8,34.075,67.650,101.725
355,2,CENTRO DE BACHILLERATO TECNOLÓGICO AGROPECUARI...,02DTA0041H,MATUTINO,EXTENSIÓN 1,BAJA CALIFORNIA,MEXICALI,POB. BENITO JUAREZ,DGETA,MEDIO,...,27.2,9.1,4.1,3.0,0.0,3.0,0.0,30.300,27.275,57.575
377,2,BELLA VISTA,02ETC0018D,VESPERTINO,EXTENSIÓN 1,BAJA CALIFORNIA,MEXICALI,MEXICALI,CECYTE,MUY BAJO,...,32.8,11.4,3.8,37.4,31.9,69.3,19.8,53.900,56.050,109.950
394,2,BELLA VISTA,02ETC0018D,MATUTINO,EXTENSIÓN 1,BAJA CALIFORNIA,MEXICALI,MEXICALI,CECYTE,MUY BAJO,...,32.8,11.4,3.8,51.1,6.8,57.9,17.8,59.425,39.150,98.575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14339,32,CENTRO DE BACHILLERATO TECNOLÓGICO AGROPECUARI...,32DTA0006W,MATUTINO,EXTENSIÓN 1,ZACATECAS,FRESNILLO,RÍO FLORIDO,DGETA,BAJO,...,30,12.8,7.1,48.0,20.0,68.0,20.0,61.000,41.000,102.000
14349,32,COLEGIO DE BACHILLERES PLANTEL GENARO CODINA,32ECB0018I,MATUTINO,EXTENSIÓN 1,ZACATECAS,GENARO CODINA,PERALES,COBACH,MEDIO,...,27.6,11.3,5.1,17.2,6.9,24.1,6.9,42.225,30.175,72.400
14356,32,COLEGIO DE BACHILLERES PLANTEL IGNACIO ZARAGOZA,32ECB0027Q,VESPERTINO,EXTENSIÓN 1,ZACATECAS,NORIA DE ANGELES,NORIA DE ANGELES,COBACH,ALTO,...,24.3,8.3,3.7,16.7,11.8,28.5,5.6,40.300,35.300,75.600
14471,21,PREP. ALFONSO CALDERÓN MORENO,21UBH0004M,MATUTINO,EXTENSIÓN 1,PUEBLA,ACAJETE,ACAJETE,BACHILLERATO AUTÓNOMO,ALTO,...,22.6,9.5,3.2,70.6,33.4,104.0,50.5,72.800,56.100,128.900


### Eliminar filas con extensión y no ser la extensión 1

+ Elimina filas con EXTENSIÓN 2 en lo adelante

In [46]:
# Eliminar filas que cumplan condición de no estar vacía la extensón y no ser la extensión 1
planea.drop(planea[(planea.EXTENSIÓN != 'NO') & (planea.EXTENSIÓN !='EXTENSIÓN 1')].index, inplace=True)
planea.shape
planea.EXTENSIÓN.value_counts()


(14568, 48)

NO             14322
EXTENSIÓN 1      246
Name: EXTENSIÓN, dtype: int64

In [47]:
planea

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
0,12,UNIDAD ACADÉMICA: PREPARATORIA NO. 36,12UBH0009Z,MATUTINO,NO,GUERRERO,EDUARDO NERI,ZUMPANGO DEL RIO,BACHILLERATO AUTÓNOMO,ALTO,...,22.6,9.5,3.2,27.6,6.4,34.0,3.8,47.600,35.775,83.375
1,1,CBTA NO. 40 JOSÉ GUADALUPE POSADA,01DTA0040J,MATUTINO,NO,AGUASCALIENTES,ASIENTOS,VILLA JUAREZ,DGETA,BAJO,...,30,12.8,7.1,44.5,16.9,61.4,16.5,56.550,43.850,100.400
2,12,UNIDAD ACADÉMICA: PREPARATORIA NO. 36,12UBH0009Z,VESPERTINO,NO,GUERRERO,EDUARDO NERI,ZUMPANGO DEL RIO,BACHILLERATO AUTÓNOMO,ALTO,...,22.6,9.5,3.2,7.2,3.5,10.7,1.8,33.100,29.825,62.925
3,12,CENTRO DE ESTUDIOS DEL BACHILLERATO GENERAL JE...,12PBH0012B,MATUTINO,NO,GUERRERO,EDUARDO NERI,ZUMPANGO DEL RIO,PARTICULARES,ALTO,...,19.7,8.2,5.1,0.0,0.0,0.0,0.0,33.325,25.000,58.325
4,1,TELEBACHILLERATO VALLADOLID,01ETH0016B,MATUTINO,NO,AGUASCALIENTES,JESUS MARIA,VALLADOLID,TELEBACHILLERATOS,MUY BAJO,...,26.9,6.1,1.2,30.0,5.0,35.0,5.0,45.000,36.250,81.250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14649,33,SISTEMA EDUCATIVO NACIONAL *10344,13DEX0001E,MATUTINO,NO,HIDALGO,PACHUCA DE SOTO,PACHUCA DE SOTO,PÚBLICO,BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.000,0.000,0.000
14650,33,INSTITUTO SONORENSE DE EDUCACION * 060911,26EEX0001H,MATUTINO,NO,SONORA,HERMOSILLO,HERMOSILLO,PÚBLICO,MUY BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.000,0.000,0.000
14651,33,COLEGIO CALMECAC *9683,14PBJ0010Q,MATUTINO,NO,JALISCO,ZAPOPAN,ZAPOPAN,PARTICULARES,MUY BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.000,0.000,0.000
14652,33,SISTEMA EDUCATIVO NACIONAL *10343,11DEX0001G,MATUTINO,NO,GUANAJUATO,CELAYA,CELAYA,PÚBLICO,BAJO,...,0,0,0,0.0,0.0,0.0,0.0,0.000,0.000,0.000


In [48]:
# Para mostrar con EXTENSIÓN 1
planea.loc[planea.EXTENSIÓN =='EXTENSIÓN 1']

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
303,2,URBIVILLA,02ETC0006Z,VESPERTINO,EXTENSIÓN 1,BAJA CALIFORNIA,TIJUANA,TIJUANA,CECYTE,MUY BAJO,...,32.8,11.4,3.8,27.7,4.9,32.6,4.9,44.750,32.125,76.875
351,2,PRIMO TAPIA ADULTOS,02ETC0016F,VESPERTINO,EXTENSIÓN 1,BAJA CALIFORNIA,PLAYAS DE ROSARITO,PRIMO TAPIA,CECYTE,BAJO,...,31,12.8,5.7,3.0,70.6,73.6,11.8,34.075,67.650,101.725
355,2,CENTRO DE BACHILLERATO TECNOLÓGICO AGROPECUARI...,02DTA0041H,MATUTINO,EXTENSIÓN 1,BAJA CALIFORNIA,MEXICALI,POB. BENITO JUAREZ,DGETA,MEDIO,...,27.2,9.1,4.1,3.0,0.0,3.0,0.0,30.300,27.275,57.575
377,2,BELLA VISTA,02ETC0018D,VESPERTINO,EXTENSIÓN 1,BAJA CALIFORNIA,MEXICALI,MEXICALI,CECYTE,MUY BAJO,...,32.8,11.4,3.8,37.4,31.9,69.3,19.8,53.900,56.050,109.950
394,2,BELLA VISTA,02ETC0018D,MATUTINO,EXTENSIÓN 1,BAJA CALIFORNIA,MEXICALI,MEXICALI,CECYTE,MUY BAJO,...,32.8,11.4,3.8,51.1,6.8,57.9,17.8,59.425,39.150,98.575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14339,32,CENTRO DE BACHILLERATO TECNOLÓGICO AGROPECUARI...,32DTA0006W,MATUTINO,EXTENSIÓN 1,ZACATECAS,FRESNILLO,RÍO FLORIDO,DGETA,BAJO,...,30,12.8,7.1,48.0,20.0,68.0,20.0,61.000,41.000,102.000
14349,32,COLEGIO DE BACHILLERES PLANTEL GENARO CODINA,32ECB0018I,MATUTINO,EXTENSIÓN 1,ZACATECAS,GENARO CODINA,PERALES,COBACH,MEDIO,...,27.6,11.3,5.1,17.2,6.9,24.1,6.9,42.225,30.175,72.400
14356,32,COLEGIO DE BACHILLERES PLANTEL IGNACIO ZARAGOZA,32ECB0027Q,VESPERTINO,EXTENSIÓN 1,ZACATECAS,NORIA DE ANGELES,NORIA DE ANGELES,COBACH,ALTO,...,24.3,8.3,3.7,16.7,11.8,28.5,5.6,40.300,35.300,75.600
14471,21,PREP. ALFONSO CALDERÓN MORENO,21UBH0004M,MATUTINO,EXTENSIÓN 1,PUEBLA,ACAJETE,ACAJETE,BACHILLERATO AUTÓNOMO,ALTO,...,22.6,9.5,3.2,70.6,33.4,104.0,50.5,72.800,56.100,128.900


In [49]:
count_claves =planea.CLAVE_ESCUELA.value_counts()
count_claves = pd.DataFrame(count_claves)
count_claves
type(count_claves)

,CLAVE_ESCUELA
12DPT0006K,4
25UBH0019K,4
11PBH3810R,3
25UBH0002K,3
14UBH0031Z,3
...,...
13PBH0002U,1
15ETC0035Y,1
15EBH0430E,1
15EBH0110U,1


pandas.core.frame.DataFrame

In [50]:
f'{len(count_claves)} claves (CCT) diferentes'

'12848 claves (CCT) diferentes'

In [51]:
count_claves.columns

Index(['CLAVE_ESCUELA'], dtype='object')

In [52]:
# Eliminar las claves de escuelas con una sola fila (un solo turno-extensión)
count_claves.drop(count_claves[count_claves.CLAVE_ESCUELA == 1].index,inplace= True)
count_claves

,CLAVE_ESCUELA
12DPT0006K,4
25UBH0019K,4
11PBH3810R,3
25UBH0002K,3
14UBH0031Z,3
...,...
14UBH0142E,2
15UBH0012K,2
05DCT0048W,2
15PBH6025R,2


### Hallazgo:
+ La mayoría de las escuelas NO aparece más de una vez. Lo más probable es que, además de las extensiones, aparezcan más de una vez por los turnos matutino y vespertino

In [53]:
count_claves[['CLAVE_ESCUELA']].value_counts()

CLAVE_ESCUELA
2                1520
3                  97
4                   2
dtype: int64

In [54]:
f'{len(count_claves)} claves (CCT) repetidas'

'1619 claves (CCT) repetidas'

In [55]:
planea.loc[planea.CLAVE_ESCUELA =='12DPT0006K',['CLAVE_ESCUELA','TURNO','EXTENSIÓN', 'ind_gral']]
# El mejor resultado lo obtuvo la matriz matutina y el peor la matriz vespertina

,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ind_gral
3724,12DPT0006K,MATUTINO,NO,81.275
3735,12DPT0006K,MATUTINO,EXTENSIÓN 1,74.950
3739,12DPT0006K,VESPERTINO,EXTENSIÓN 1,65.275
3742,12DPT0006K,VESPERTINO,NO,61.125


### Hallazgo:

+ La matriz tiene dos turnos y la extensión 1 también. Nos quedaremos con la matriz matutina

In [56]:
planea.loc[planea.CLAVE_ESCUELA =='25UBH0019K',['CLAVE_ESCUELA','TURNO','EXTENSIÓN', 'ind_gral']] 
# El mejor resultado lo obtuvo la matriz matutina y el peor la extensión vespertina

,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ind_gral
8155,25UBH0019K,MATUTINO,EXTENSIÓN 1,55.650
8156,25UBH0019K,VESPERTINO,EXTENSIÓN 1,52.450
10005,25UBH0019K,MATUTINO,NO,97.425
10010,25UBH0019K,VESPERTINO,NO,66.050


In [57]:
planea.loc[planea.CLAVE_ESCUELA =='14UBH0015I',['CLAVE_ESCUELA','TURNO','EXTENSIÓN', 'ind_gral']] 
# Matriz con los 3 turnos y el mejor resultado en el matutino

,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ind_gral
13622,14UBH0015I,MATUTINO,NO,104.275
13623,14UBH0015I,VESPERTINO,NO,94.575
13626,14UBH0015I,NOCTURNO,NO,74.600


In [58]:
planea.loc[planea.CLAVE_ESCUELA =='21UBH0038C',['CLAVE_ESCUELA','TURNO','EXTENSIÓN', 'ind_gral']]
# Mejor resultado la extensión matutina

,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ind_gral
3919,21UBH0038C,MATUTINO,EXTENSIÓN 1,167.925
5585,21UBH0038C,VESPERTINO,NO,162.375
5587,21UBH0038C,MATUTINO,NO,153.750


In [59]:
planea.loc[planea.CLAVE_ESCUELA =='29DCT0144I',['CLAVE_ESCUELA','TURNO','EXTENSIÓN', 'ind_gral']]

,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ind_gral
11824,29DCT0144I,MATUTINO,NO,133.30
11825,29DCT0144I,VESPERTINO,NO,97.95


### Hallazgo
+ Las repeticiones son porque hay turnos matutinos y vespertinos, además de la extensión
+ No siempre la matriz o el turno matutino tienen el mejor ind_gral

## 3. Eliminar CCT duplicados con menor ind_gral 

+ Sin importar si es matriz o EXTENSIÓN 1
+ Localizar filas con esa condición 
+ Ordenar y quedarse con fila de mayor ind_gral

In [60]:
print(len(planea))

14568


In [61]:
planea.loc[planea['CLAVE_ESCUELA'] == '31DTA0013G']

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
14047,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 1,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,22.6,3.2,25.8,4.3,44.125,33.8,77.925


In [62]:
# Para cada CCT (algo más de 1600)
for i in count_claves.index:
    f = planea.loc[(planea.CLAVE_ESCUELA == i)] # Localiza registros de PLANEA con igual CCT. únicos,duplicados, triplicados y más
    ordenado = f.sort_values('ind_gral') # ordenado de menor índice general a mayor de un mismo CCT
    ordenado.drop(ordenado.tail(1).index,inplace=True)  # Elimina el último de mayor ind_gral
    # Elimina las filas de ordenado (con los índices que no son menores) en planea
    planea.drop(ordenado.index,inplace=True)
        
print(len(planea))

12848


In [63]:
planea.loc[planea['CLAVE_ESCUELA'] == '31DTA0013G']

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral
14047,31,CBTA # 13,31DTA0013G,MATUTINO,EXTENSIÓN 1,YUCATAN,XMATKUIL,XMATKUIL,DGETA,ALTO,...,26.9,11.4,6.3,22.6,3.2,25.8,4.3,44.125,33.8,77.925


### Exploración de ALUMNOS_CONTESTARON_50_PORCIENTO_O_MÁS_PREG_MAT

In [64]:
planea.ALUMNOS_CONTESTARON_50_PORCIENTO_O_MÁS_PREG_MAT.value_counts()

16     288
13     268
17     267
19     263
15     259
      ... 
466      1
606      1
448      1
434      1
447      1
Name: ALUMNOS_CONTESTARON_50_PORCIENTO_O_MÁS_PREG_MAT, Length: 465, dtype: int64

In [65]:
planea[planea['ALUMNOS_CONTESTARON_50_PORCIENTO_O_MÁS_PREG_MAT'].isin([np.nan, np.inf, -np.inf])]


,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral


### Nueva verificación de nulos

In [66]:
# Cantidad de filas con al menos un nan. Es lento
#sum(planea.apply(lambda x: sum(x.isnull().values), axis = 1)>0)
# Hubo null antes de imputar extensión y cambiar a bool'ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO'
# Después ya no hay nulos

In [67]:
# Otra forma de hacer lo anterior de manera más rápida
# Cantidad de filas con al menos un valor faltante
(planea.isna().sum(axis=1) > 0).sum()

0

In [68]:
# Cantidad e columnas con al menos un valor faltante
(planea.isna().sum(axis=0) > 0).sum()

0

In [69]:
planea_nan = planea[planea.isna().any(axis=1)]
planea_nan

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral


### Convertir a tipos de menos memoria

+ Usando dict comprehension o de uno por uno
+ Memoria usada antes y después de la reducción

In [70]:
# Usando un dict por comprensión para cambiar todos los de un mismo tipo
#df_mod = df_mod.astype({col: 'int32' for col in df_mod.select_dtypes('int64').columns})  # dict por comprensión

In [71]:
planea.memory_usage(index=True,deep=False).sum()/1024

4918.375

In [72]:
# Aquí se hace de uno por uno
planea['CLAVE DE LA ENTIDAD'] = planea['CLAVE DE LA ENTIDAD'].astype('int32')
planea['ALUMNOS_PROGRAMADOS'] = planea['ALUMNOS_PROGRAMADOS'].astype('int32')
planea['ALUMNOS_EVALUADOS'] = planea['ALUMNOS_EVALUADOS'].astype('int32')
planea['ALUMNOS_CONTESTARON_ 50_PORCIENTO_O_MÁS_PREG_LENGUAJE'] = planea['ALUMNOS_CONTESTARON_ 50_PORCIENTO_O_MÁS_PREG_LENGUAJE'].astype('int32')
planea['ALUMNOS_CONTESTARON_50_PORCIENTO_O_MÁS_PREG_MAT'] = planea['ALUMNOS_CONTESTARON_50_PORCIENTO_O_MÁS_PREG_MAT'].astype('int32')

In [73]:
planea.memory_usage(index=True,deep=False).sum()/1024

4667.4375

### Verificar nulos y tipos

In [74]:
planea.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12848 entries, 0 to 14653
Data columns (total 48 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   CLAVE DE LA ENTIDAD                                    12848 non-null  int32  
 1   NOMBRE_ESCUELA                                         12848 non-null  object 
 2   CLAVE_ESCUELA                                          12848 non-null  object 
 3   TURNO                                                  12848 non-null  object 
 4   EXTENSIÓN                                              12848 non-null  object 
 5   ENTIDAD                                                12848 non-null  object 
 6   MUNICIPIO                                              12848 non-null  object 
 7   LOCALIDAD                                              12848 non-null  object 
 8   SUBSISTEMA                                    

## 4. Imputar con 0 donde aparece 'S/R' (sin respuesta)

In [75]:
# Localizar donde hay 'S/R'
planea.loc[planea.ent_ss_mar_mat_I == 'S/R',['ent_ss_mar_len_I','ent_ss_mar_mat_I',
                                             'pa_ss_mar_mat_I','pa_ss_mar_mat_II','pa_ss_mar_mat_III','pa_ss_mar_mat_IV']]

,ent_ss_mar_len_I,ent_ss_mar_mat_I,pa_ss_mar_mat_I,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV
37,S/R,S/R,S/R,S/R,S/R,S/R
39,S/R,S/R,64.3,22.1,9.5,4.2
54,S/R,S/R,S/R,S/R,S/R,S/R
56,S/R,S/R,60.9,26,9.1,4
65,S/R,S/R,55.1,28.6,11.6,4.7
...,...,...,...,...,...,...
14371,S/R,S/R,57.7,29.3,8.9,4.1
14468,S/R,S/R,59.6,27.2,9.1,4.1
14483,S/R,S/R,51.6,27.3,16.5,4.7
14496,S/R,S/R,54.2,31.7,10,4


### Hallazgo
+ Hay 193 filas con valores de S/R

In [76]:
# Reemplazar todos los S/R
planea.replace('S/R',0,inplace=True)

In [77]:
planea.loc[planea.ent_ss_mar_mat_I == 'S/R',['ent_ss_mar_len_I','ent_ss_mar_mat_I',
                                             'pa_ss_mar_mat_I','pa_ss_mar_mat_II','pa_ss_mar_mat_III','pa_ss_mar_mat_IV']]

,ent_ss_mar_len_I,ent_ss_mar_mat_I,pa_ss_mar_mat_I,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV


In [78]:
planea.esc_mat_I.value_counts()

100.0    496
0.0      436
50.0     303
66.7     261
75.0     234
        ... 
60.1       1
11.2       1
93.0       1
89.6       1
50.2       1
Name: esc_mat_I, Length: 962, dtype: int64

In [79]:
planea.loc[planea.esc_mat_I == '*****']

,CLAVE DE LA ENTIDAD,NOMBRE_ESCUELA,CLAVE_ESCUELA,TURNO,EXTENSIÓN,ENTIDAD,MUNICIPIO,LOCALIDAD,SUBSISTEMA,GRADO_MARGINACIÓN_LOCALIDAD_CCT,...,pa_ss_mar_mat_II,pa_ss_mar_mat_III,pa_ss_mar_mat_IV,ind_len,ind_mat,ind-compuesto,ind_IV,ind_gral_len,ind_gral_mat,ind_gral


In [80]:
planea.loc[planea.ent_ss_mar_mat_I == 'S/R',['ent_ss_mar_len_I','ent_ss_mar_mat_I']]

,ent_ss_mar_len_I,ent_ss_mar_mat_I


### Reducir de float64 a float32

In [81]:
lista = ['esc_leng_I', 'esc_leng_II', 'esc_leng_III', 'esc_leng_IV',
       'ent_ss_mar_len_I', 'ent_ss_mar_len_II', 'ent_ss_mar_len_III',
       'ent_ss_mar_len_IV', 'pa_ss_mar_len_I', 'pa_ss_mar_len_II',
       'pa_ss_mar_len_III', 'pa_ss_mar_len_IV', 'esc_mat_I', 'esc_mat_II',
       'esc_mat_III', 'esc_mat_IV', 'ent_ss_mar_mat_I', 'ent_ss_mar_mat_II',
       'ent_ss_mar_mat_III', 'ent_ss_mar_mat_IV', 'pa_ss_mar_mat_I',
       'pa_ss_mar_mat_II', 'pa_ss_mar_mat_III', 'pa_ss_mar_mat_IV']
for i in lista:
    planea[i] = planea[i].astype('float32')


In [82]:
planea.memory_usage(index=True,deep=False).sum()/1024

3375.109375

In [83]:
planea.dtypes

CLAVE DE LA ENTIDAD                                        int32
NOMBRE_ESCUELA                                            object
CLAVE_ESCUELA                                             object
TURNO                                                     object
EXTENSIÓN                                                 object
ENTIDAD                                                   object
MUNICIPIO                                                 object
LOCALIDAD                                                 object
SUBSISTEMA                                                object
GRADO_MARGINACIÓN_LOCALIDAD_CCT                           object
SOSTENIMIENTO                                             object
ALUMNOS_PROGRAMADOS                                        int32
ALUMNOS_EVALUADOS                                          int32
PORCENTAJE_ ALUMNOS_EVALUADOS                            float64
ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO                        bool
ALUMNOS_CONTESTARON_ 50_P

In [84]:
# Nominales de baja cardinalidad
categoricas = ['CLAVE DE LA ENTIDAD', 'NOMBRE_ESCUELA', 'CLAVE_ESCUELA', 'TURNO',
       'EXTENSIÓN', 'ENTIDAD', 'MUNICIPIO', 'LOCALIDAD', 'SUBSISTEMA',
       'GRADO_MARGINACIÓN_LOCALIDAD_CCT', 'SOSTENIMIENTO']
nominal_low_card = ['TURNO','EXTENSIÓN', 'GRADO_MARGINACIÓN_LOCALIDAD_CCT', 'SOSTENIMIENTO']

In [85]:
planea.SUBSISTEMA.unique()

array(['BACHILLERATO AUTÓNOMO', 'DGETA', 'PARTICULARES',
       'TELEBACHILLERATOS', 'EMSAD', 'ASOCIACIÓN CIVIL', 'CECYTE',
       'TELEBACHILLERATOS COMUNITARIO', 'DGETI', 'DGB',
       'BACH ESTATAL DGE-CGE', 'CONALEP', 'COBACH', 'DGECYTM',
       'OTRAS FEDERALES', 'OTRAS ESTATALES', 'COLBACH MEX', 'CETI',
       'EXTRANJERO', 'REVALIDACION', 'PÚBLICO'], dtype=object)

In [86]:
planea.GRADO_MARGINACIÓN_LOCALIDAD_CCT.unique()

array(['ALTO', 'BAJO', 'MUY BAJO', 'MEDIO', 'MUY ALTO'], dtype=object)

In [87]:
planea.SOSTENIMIENTO.unique()

array(['AUTÓNOMAS', 'FEDERAL', 'PARTICULARES', 'ESTATAL', 'EXTRANJERO'],
      dtype=object)

In [88]:
planea.TURNO.unique()

array(['MATUTINO', 'VESPERTINO', 'NOCTURNO', 'DISCONTINUO'], dtype=object)

In [89]:
pd.get_dummies?

In [90]:
planea_dummies =pd.get_dummies(planea[nominal_low_card],drop_first=True)
planea_dummies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12848 entries, 0 to 14653
Data columns (total 12 columns):
 #   Column                                    Non-Null Count  Dtype
---  ------                                    --------------  -----
 0   TURNO_MATUTINO                            12848 non-null  uint8
 1   TURNO_NOCTURNO                            12848 non-null  uint8
 2   TURNO_VESPERTINO                          12848 non-null  uint8
 3   EXTENSIÓN_NO                              12848 non-null  uint8
 4   GRADO_MARGINACIÓN_LOCALIDAD_CCT_BAJO      12848 non-null  uint8
 5   GRADO_MARGINACIÓN_LOCALIDAD_CCT_MEDIO     12848 non-null  uint8
 6   GRADO_MARGINACIÓN_LOCALIDAD_CCT_MUY ALTO  12848 non-null  uint8
 7   GRADO_MARGINACIÓN_LOCALIDAD_CCT_MUY BAJO  12848 non-null  uint8
 8   SOSTENIMIENTO_ESTATAL                     12848 non-null  uint8
 9   SOSTENIMIENTO_EXTRANJERO                  12848 non-null  uint8
 10  SOSTENIMIENTO_FEDERAL                     12848 non-null  

## Cambiar a tipo category las columnas nominales de tipo object que se necesitan como variables para la analítica

+ Las de cardinalidad 1 (son llaves candidas de la relación, no se necesitan convertir)

In [91]:
for col in nominal_low_card:
    planea[col] = planea[col].astype('category')

In [92]:
planea.dtypes

CLAVE DE LA ENTIDAD                                         int32
NOMBRE_ESCUELA                                             object
CLAVE_ESCUELA                                              object
TURNO                                                    category
EXTENSIÓN                                                category
ENTIDAD                                                    object
MUNICIPIO                                                  object
LOCALIDAD                                                  object
SUBSISTEMA                                                 object
GRADO_MARGINACIÓN_LOCALIDAD_CCT                          category
SOSTENIMIENTO                                            category
ALUMNOS_PROGRAMADOS                                         int32
ALUMNOS_EVALUADOS                                           int32
PORCENTAJE_ ALUMNOS_EVALUADOS                             float64
ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO                         bool
ALUMNOS_CO

## 5. Guardar archivo de PLANEA 2015 limpio y sus tipos

In [93]:
# DateFrame con dtypes que no son datetime

tipos = pd.DataFrame(planea.dtypes,columns=['dtypes'])

non_date = tipos[tipos['dtypes'] != 'datetime64[ns]']
non_date

,dtypes
CLAVE DE LA ENTIDAD,int32
NOMBRE_ESCUELA,object
CLAVE_ESCUELA,object
TURNO,category
EXTENSIÓN,category
ENTIDAD,object
MUNICIPIO,object
LOCALIDAD,object
SUBSISTEMA,object
GRADO_MARGINACIÓN_LOCALIDAD_CCT,category


In [95]:
# Metadatos
# Guardar en archivo los tipos que no son datetime
non_date.to_csv( 'tipos_Planea2015.csv',index_label='columnas',header=['tipos'],encoding='latin1')

In [97]:
planea.to_csv('Planea2015_Limpio_newFeatures.csv',header=True,index=False,encoding='latin')

In [98]:
len(planea)

12848

In [99]:
planea.isnull().sum()

CLAVE DE LA ENTIDAD                                      0
NOMBRE_ESCUELA                                           0
CLAVE_ESCUELA                                            0
TURNO                                                    0
EXTENSIÓN                                                0
ENTIDAD                                                  0
MUNICIPIO                                                0
LOCALIDAD                                                0
SUBSISTEMA                                               0
GRADO_MARGINACIÓN_LOCALIDAD_CCT                          0
SOSTENIMIENTO                                            0
ALUMNOS_PROGRAMADOS                                      0
ALUMNOS_EVALUADOS                                        0
PORCENTAJE_ ALUMNOS_EVALUADOS                            0
ALUMNOS_EVALUADOS_MENOR_80_PORCIENTO                     0
ALUMNOS_CONTESTARON_ 50_PORCIENTO_O_MÁS_PREG_LENGUAJE    0
ALUMNOS_CONTESTARON_50_PORCIENTO_O_MÁS_PREG_MAT         